# Imports

In [1]:
import sys

import optuna

sys.path.append("..")

from constants import U1BASE_PATH
from benchmark.data_tramsformation import split_raw_to_train_test
from models.models import ColaborativeModel
from benchmark.count_metrics import count_rmse_on_dataset

# Data

In [2]:
train, valid = split_raw_to_train_test(U1BASE_PATH)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


udata = pd.read_csv(U1BASE_PATH, sep="\t", header=None)
udata.columns = ["user_id", "item_id", "rating", "timestamp"]

udata = np.array(udata)
udata = udata[:, :3]

train, test = train_test_split(udata, test_size=0.2)

train.shape

(64000, 3)

# Hyperparameters tunning

In [6]:
def objective(trial):
    num_users = trial.suggest_int("num_users", 1, 900)
    colab_model = ColaborativeModel(num_users=num_users)
    colab_model.fit(train)

    rmse = count_rmse_on_dataset(colab_model, valid)
    
    return rmse

In [7]:
study = optuna.create_study(direction='minimize', study_name="Best number of uzers")
study.optimize(objective, n_trials=5)

[I 2023-12-03 22:12:01,596] A new study created in memory with name: Best number of uzers
[W 2023-12-03 22:12:01,866] Trial 0 failed with parameters: {'num_users': 772} because of the following error: IndexError('boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 1682').
Traceback (most recent call last):
  File "c:\Users\Andrey\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\Andrey\AppData\Local\Temp\ipykernel_26304\1234051082.py", line 6, in objective
    rmse = count_rmse_on_dataset(colab_model, valid)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Studying\Practical Machine Learning\assignment_2\movie-recommender-system\notebooks\..\benchmark\count_metrics.py", line 24, in count_rmse_on_dataset
    rmse = count_rmse(sample, pred)
           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Studying\Pr

IndexError: boolean index did not match indexed array along dimension 0; dimension is 3 but corresponding boolean dimension is 1682

In [ ]:
#!c1.32
print("\nBest trial:")
trial = study.best_trial
print("  RMSE score: ", trial.value)
print("\n  Params: ")

for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Best trial:
  RMSE score:  2.512494308947912

  Params: 
    num_users: 20
